In [ ]:
import pandas as pd

from datetime import datetime
# Display figures inline in Jupyter notebook
import matplotlib.pyplot as plt
import seaborn as sn
# Use seaborn style defaults and set the default figure size
sn.set(rc={'figure.figsize':(11, 4)})


In [ ]:
# !pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

# Reading  the data files

In [ ]:
events=pd.read_csv('events.csv')

In [ ]:
events.head(5)

In [ ]:
category_tree=pd.read_csv('category_tree.csv')

In [ ]:
category_tree.head(5)

In [ ]:
category_tree.shape

In [ ]:
category_tree.parentid.nunique()

In [ ]:
item_prop=pd.read_csv('item_properties_part1.csv')
item_prop2=pd.read_csv('item_properties_part2.csv')

In [ ]:
item_props=pd.concat([item_prop,item_prop2],ignore_index=True)

In [ ]:
item_props.head(5)

In [ ]:
item_props.dtypes

In [ ]:
item_props=item_props.sort_values(by='timestamp')

Change timestamp to date-time format i tried to convert it directly but it gave me DateTime in the future so i divided the value by 1000 before converting it to be reasonable. maybe it's  wrong but I did this just to be  readable

In [ ]:
item_props['timestamp']=item_props['timestamp'].apply( lambda x: datetime.fromtimestamp(x/1000))

In [ ]:
item_props.shape

In [ ]:
item_props.itemid.nunique()

In [ ]:
item_props

# Exploring  properties of  items  over time

In [ ]:
item_props=item_props.set_index('timestamp')

In [ ]:
item_props=item_props.sort_values(by=['timestamp', 'itemid','property', 'value'])

In [ ]:
item_props

#  Most frequently changed  properties 


from the  below  code  we  find  properties  <b>888,790,available,categoryid,451 </b>the most frequently changed  properties

In [ ]:
# item_props[item_props.duplicated(['itemid','property'])].property.value_counts()

In [ ]:

sn.barplot(x=item_props.property.value_counts().index[0:10], y=item_props.property.value_counts()[0:10])

get the changed items over time 

In [ ]:
changed_items=item_props[item_props.duplicated(['itemid','property'])]
changed_items

# listing the items that have different category over time

In [ ]:
changed_items[changed_items['property']=='categoryid']

let's check  frist item <b> 393623</b> and property <b>categoryid</b>

In [ ]:
# item_props[(item_props['itemid']==369722) &(item_props['property']=='categoryid')]['value']

In [ ]:
item_props[(item_props['itemid']==393623) &(item_props['property']=='categoryid')]['value'].astype(int).plot(grid='True')

# the changes in items avaiablibty over time

In [ ]:
changed_items[changed_items['property']=='available']

checck  avaiablibty of item 267142 and 379701 over time 

In [ ]:
item_props[(item_props['itemid']==267142) &(item_props['property']=='available')]['value'].astype(int).plot(grid='True')

In [ ]:
item_props[(item_props['itemid']==379701) &(item_props['property']=='available')]['value'].astype(int).plot(grid='True')

In [ ]:
# item_props[(item_props['itemid']==267142) &(item_props['property']=='available')]['value']

item 267142 almost not available most of time it was available only once at 2015-08-02 05:00:00 in contrast to  item  379701 which was avaiable most  of time 

# Num of  unique properties per item

In [ ]:
propertiesperitem=item_props.groupby(['itemid'])['property'].nunique()

In [ ]:
propertiesperitem=propertiesperitem.to_frame( name="nuniqueproperty")

In [ ]:
propertiesperitem

from the  below output we find that the  averaage  number of properties  per item amlost <b>28</b> and  max is  <b>59</b> and  min <b>12 </b>

In [ ]:
propertiesperitem.describe()

In [ ]:
# item_props.groupby(['itemid','property'])['value'].nunique().to_frame(name='numofchanges').sort_values(by='numofchanges',ascending=False)

# Properties that are constant over time

In [ ]:
valuesperproperty=item_props.groupby(['itemid','property'],as_index=False).agg({'value':{'count','nunique'}})

In [ ]:
# valuesperproperty=valuesperproperty.reset_index()

In [ ]:
valuesperproperty.columns = ["itemid","property","value_nunique","value_count"]

In [ ]:
valuesperproperty[(valuesperproperty['value_nunique']==1)&(valuesperproperty['value_count']>1)]

<b>Validating resullt by explore example of returned rows</b>
 <li> we can see rom the  below output /graph that the  value is the same  over time 
    

In [ ]:
item_props[(item_props.itemid==40)&(item_props.property=='810')]

In [ ]:
item_props[(item_props.itemid==40)&(item_props.property=='810')]['value'].value_counts().plot.bar(stacked=True)


<li> from the  below code we can find that there are <b>1104 </b> unique property that are  constant over time for some items
<li> also we  see that the count for some  property is the same of the number of unquie items so i think this properties may be  the meta data about every item like <b>descibtion or registerdate</b>  because every item have it
 <li> the  top of this properties are <b> 764,159,790,364 and 283

In [ ]:
valuesperproperty.property.nunique()

In [ ]:
valuesperproperty.property.value_counts()

In [ ]:
item_props.itemid.nunique()

In [ ]:
sn.barplot(x=valuesperproperty.property.value_counts().index[0:20], y=valuesperproperty.property.value_counts()[0:20])

# Snapshot merge

check the dublicated snapshot 
we have <b>7497165</b> dublicated rows of <B> 20275902 </b> which will reduce the data size  by <b> 40% </b> after droping it

In [ ]:
item_props[item_props.duplicated(['itemid','property','value'])]

Explore item <b>466864</b> with property <b>813</b> and value<b> '1148082 353870 1262739'</b> snapshots
from the below graph we note that this property had been changed once for this  item.

In [ ]:
item1=item_props[(item_props.itemid==466864)&(item_props.property=='813')]
item1.groupby(['itemid','property'])['value'].value_counts().unstack().plot.bar(stacked=True)

let's drop the duplicates with consider keeping the last snapshot

In [ ]:
item_props_merged=item_props.drop_duplicates(subset=['itemid','property','value'], keep='last')

In [ ]:
item_props_merged.shape

# Checking some items before and  after  snapshots merge

Before Merge

In [ ]:
item_props[(item_props.itemid==91477)&(item_props.property=='888')]

After merge

In [ ]:
item_props_merged[(item_props_merged.itemid==91477)&(item_props_merged.property=='888')]

Before merge 

In [ ]:
item_props[(item_props.itemid==89439)&(item_props.property=='888')]

In [ ]:
item_props_merged[(item_props_merged.itemid==89439)&(item_props_merged.property=='888')]

# Exploring  events data

<li>we have <b>2756101</b> row in event data  and <b>5 </b>columns
<li><b>1407580</b> unique vistior
<li><b>235061</b> unique items 
<li> <b>3 </b>types of events view,addtocart and transaction
<li> for the below  graph the most occured event is view by precentage <b> 96%</b> events. then addtocart and the lowest is transaction with  precentang <b>0.8% 
   

In [ ]:
events

Change timestamp to date-time format i tried to convert it directly but it gave me DateTime in the future so i divided the value by 1000 before converting it to be reasonable. maybe it's  wrong but I did this just to be  readable

In [ ]:
events['timestamp']=events['timestamp'].apply( lambda x: datetime.fromtimestamp(x/1000))

In [ ]:
events=events.set_index('timestamp')
events=events.sort_values(by='timestamp')

In [ ]:
events.visitorid.nunique()

In [ ]:
events.itemid.nunique()

In [ ]:
events.event.describe()

In [ ]:
events.event.value_counts().plot.bar(stacked=True)

## Validating each row that has event type transaction have already transaction-id

In [ ]:
events[(events.event=='transaction')&(events.transactionid.isna())]

In [ ]:
events[events.event=='transaction']

#  Transcations per visitor

<li> number of visitors who are purchasing is <b>11719</b>
<li> the visitor who has more transactions is <b>1150086</b> he has <b>559</b>
<li> the  avg  number of transcation per  vistor  is  <b>~2 </b>and min  is  <b>1 </b> and  max is <b>559</b>

In [ ]:
events[events.event=='transaction']['visitorid'].nunique()

In [ ]:
transpervistior=events[events.event=='transaction'].groupby('visitorid').agg({'transactionid':'count'})
transpervistior.columns=['totalnumtrans']

In [ ]:
transpervistior=transpervistior.sort_values(by='totalnumtrans',ascending=False)

In [ ]:
transpervistior

In [ ]:
transpervistior.describe()

In [ ]:
sn.barplot(x=transpervistior.index[0:15], y=transpervistior.totalnumtrans[0:15] )

# Items  per transaction

<li> max number of items  per transcation is <b>31</b> and min is <b>1</b> and avg is  almost also<b> 1 </b>
<li>total number  of items Purchased is highest in half of the months

In [ ]:
itemspertrans=events.reset_index().groupby('transactionid',as_index=False).agg({'timestamp':lambda x:max(x),'itemid':['unique','count']})
itemspertrans.columns=['transactionid','timestamp','unique_items','totalnumitems']
itemspertrans=itemspertrans.sort_values(by='totalnumitems',ascending=False)

In [ ]:
itemspertrans

In [ ]:
itemspertrans.totalnumitems.describe()

In [ ]:
itemspertrans.set_index('timestamp')['totalnumitems'].plot(linewidth=0.5);

# Most Purchased items

<li> top pruchased item <b>461686</b> found  in <b>133</b> transcation

In [ ]:
trasactions=events[events.event=='transaction']

In [ ]:
trasactions

In [ ]:
trasactions['itemid'].value_counts()

In [ ]:
sn.barplot(x=trasactions.itemid.value_counts().index[0:15], y=trasactions.itemid.value_counts()[0:15] )

# Add to cart Events

<li> almost 2.5% of  events is add to cart type
 <li> highest addtocart item <b>461686</b> found  in <b>306</b> transcation which also highest purchased makesense
 <li> the below figure have  similar distribution to the above one of the purchased items because each item before being transaction it was addtocart

In [ ]:
addtocart=events[events.event=='addtocart']
addtocart

In [ ]:
addtocart['itemid'].value_counts()

In [ ]:
sn.barplot(x=addtocart.itemid.value_counts().index[0:15], y=addtocart.itemid.value_counts()[0:15] )

# Machine Learning applications


<li>Cross-selling using assoication rules 
<li>Demand Forecasting for items or forcasting  weekly sales using time series algorithms
<li>Customer Segmation using kmean cluster  and  RFM
<li> item category classifcation 

In [ ]:
transcationevents=events.dropna()
transcationevents

# Associaciton Rule Learning



The association rules method is to present these correlations in the best way through
rules, if there are significant correlations between the items that occur simultaneously
and frequently, and if there are significant correlations. In other words, it is a
rule-based machine learning technique used to find patterns in data.
       
       - Apriori Algorithm -

It is a basket analysis method and is used to reveal product associations.

Support(X, Y) = Freq(X, Y) / N

There are 3 very simple formulas. The 1st is the Support value. It expresses the probability of
X and Y occurring together. It is the frequency of X and Y appearing together divided by N.

Confidence(X, Y) = Freq(X, Y) / Freq(X)

It expresses the probability of purchasing product Y when product X is purchased.
The frequency at which X and Y appear together divided by the frequency at which X appears.

Lift = Support(X, Y) / (Support(x) * Support (Y))

When X is purchased, the probability of buying Y increases by a multiple of lift.
The probability of X and Y appearing together is the product of the probabilities
of X and Y appearing separately.
It states an expression such as how many times the probability of buying another product
increases when we buy a product.

Our aim is to suggest products to users in the product purchasing process by
applying association analysis to the online retail II dataset.

# Build Transaction Dataset


In [ ]:

    # We need to create below structure:

    # Rows represents transactions (transcation, shopping cart etc.), columns represents items
    # We simulate as binary that which transaction  contains which items
    # If the item is in the transcation, the intersection cell will be "1". If is not, it will be "0"

    # Description   Item1      Item2    Item3
    # trans
    # trans1          0         1         0
    # trans2          1         0         1
    # trans3          0         0         0
    # trans4          1         0         0
    # trans5          0         0         1


Extract Quantity of Item per transcation

In [ ]:
transcationevents.reset_index().groupby(['transactionid','itemid']).agg({"timestamp": "count"}).rename(columns={'timestamp':'Quantity'}).sort_values('Quantity',ascending=False)

Check  first item from  event  data will  find  4  rows  form same  item

In [ ]:
events[(events.transactionid==2212.0)&(events.itemid==325772)]

In [ ]:

items = list(transcationevents.itemid.unique())
grouped = transcationevents.groupby('transactionid')
transaction_level_df = grouped.aggregate(lambda x: tuple(x)).reset_index()[['transactionid','itemid']]

In [ ]:
transaction_level_df

In [ ]:
# transaction_dict = {item:0 for item in items}
# output_dict = dict()
# temp = dict()
# for rec in transaction_level_df.to_dict('records'):
#     transactionid = rec['transactionid']
#     items_list = rec['itemid']
#     transaction_dict = {item:0 for item in items}
#     transaction_dict.update({item:1 for item in items if item in items_list})
#     temp.update({transactionid:transaction_dict})

# new = [v for k,v in temp.items()]
# tranasction_df = pd.DataFrame(new)
# del(tranasction_df[tranasction_df.columns[0]])

In [ ]:

def create_transaction_item_df(df):
    return df.reset_index().groupby(['transactionid','itemid']).agg({"timestamp": "count"}).rename(columns={'timestamp':'Quantity'}).sort_values('Quantity',ascending=False).unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
gr_tran_itm_df=create_transaction_item_df(transcationevents)

In [ ]:
df=gr_tran_itm_df.copy()

In [ ]:
df.columns=[ col[1] for col in df.columns]

In [ ]:
df=df[list(trasactions.itemid.value_counts().index[0:1000])]

In [ ]:
df.head()

In [ ]:
gr_tran_itm_df.shape

In [ ]:
# 3. Possibilities of All Possible Item Combinations

# Support(X, Y) = Freq(X, Y) / N
# There are 3 very simple formulas. The 1st is the Support value. It expresses the probability of
# X and Y occurring together. It is the frequency of X and Y appearing together divided by N.

# Confidence(X, Y) = Freq(X, Y) / Freq(X)
# It expresses the probability of purchasing item Y when item X is purchased.
# The frequency at which X and Y appear together divided by the frequency at which X appears.

# Lift = Support(X, Y) / (Support(x) * Support (Y))
# When X is purchased, the probability of buying Y increases by a multiple of lift.
# The probability of X and Y appearing together is the item of the probabilities
# of X and Y appearing separately.
# It states an expression such as how many times the probability of buying another item
# increases when we buy a item.

In [ ]:

frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False).head()

In [ ]:
# By inserting the support values we found with Apriori into the association_rules function,
# we find some other statistical data such as cofidance and lift.
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.001)

rules.sort_values("support", ascending=False).head()

 According to this table, the probability of 213834 item and product numbered 445351
appearing together is 0.002207. The probability of being bought together is 0.423913.
The increase in the probability of buying these two products together is 166.475362

# Forcasting weekly sales 

In [ ]:
transcationevents["Week"]=transcationevents.index.isocalendar().week
transcationevents["Month"]=transcationevents.index.month

In [ ]:
# weekly_transcations=transcationevents.groupby(['Week','Month'],as_index=False).agg({"transactionid":'count'})

In [ ]:
weekly_transcations=transcationevents.reset_index().groupby(['Week','Month'],as_index=False).agg({"transactionid":'count','timestamp':lambda x:x.max()})

In [ ]:
# transcationevents.resample('W').mean().

In [ ]:
weekly_transcations.columns=['Week','Month','Num_trans','timestamp']

weekly_transcations

In [ ]:
weekly_transcations=weekly_transcations.set_index('timestamp').resample('W').mean()

In [ ]:
weekly_transcations

# Visual  weekly Sales time series

In [ ]:
weekly_transcations.groupby('Week').agg({'Num_trans':'sum'}).plot()

In [ ]:
sn.barplot(x=weekly_transcations['Week'], y=weekly_transcations['Num_trans'] )

# Decomposing

Seasonal Decompose gives the decomposition of the time series into its estimated trend component, estimated seasonal component, and estimated residual. We can also plot the original data to look at what components of the data influence its true value the most.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# weekly_transcations.sort_index(inplace=True)

In [ ]:
multi_plot = seasonal_decompose(weekly_transcations['Num_trans'], model = 'add', extrapolate_trend='freq', period=4)

plt.figure(figsize=(20,5))
multi_plot.observed.plot(title = 'weekly sales')

plt.figure(figsize=(20,5))
multi_plot.trend.plot(title = 'trend')

plt.figure(figsize=(20,5))
multi_plot.seasonal.plot(title = 'seasonal')

plt.figure(figsize=(20,5))
multi_plot.resid.plot(title = 'residual');

# Forecasting using ARIMA Model


In [ ]:
import statsmodels.api as sm
from math import sqrt


In [ ]:
weekly_transcations

 We will fit an ARIMA model to the entire weekly_transcations dataset and review the residual errors.
First, we fit an ARIMA(1,1,0) model. This sets the lag value to 1 for autoregression, uses a difference order of 1 to make the time series stationary, and uses a moving average model of 0.

In [ ]:
# fit model
model =sm.tsa.ARIMA(weekly_transcations['Num_trans'], order=(1,1,0))
model_fit = model.fit()
# summary of fit model
print(model_fit.summary())
# line plot of residuals
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()
# # density plot of residuals
# residuals.plot(kind='kde')
# plt.show()
# # summary stats of residuals
# print(residuals.describe())

# Split data to train and  test and  get  RMSE

Running the example prints the prediction and expected value each iteration.

We can also calculate a final root mean squared error score (RMSE) for the predictions

A line plot is created showing the expected values (blue) compared to the rolling forecast predictions (red). We can see the values show some trend and are in the correct scale.



In [ ]:
from sklearn.metrics import mean_squared_error
# split into train and test sets
X = weekly_transcations.Num_trans.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
# walk-forward validation
for t in range(len(test)):
	model = sm.tsa.ARIMA(history, order=(1,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)
# plot forecasts against actual outcomes
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()